In [2]:
#this creates a mask of the positron image to overlay on top of the CT slice
# any pixel with RBG values greater than (30,30,30) will be white, all others black

from PIL import Image

img = Image.open('Images/Pos_axial.png')
img = img.convert("RGBA")
datas = img.getdata()

RGB1 = 50
RGB2 = 50
RGB3 = 50

newData = []
for item in datas:
    if item[0] > RGB1 or item[1] > RGB2 or item[2] > RGB3:
        newData.append((255, 255, 255, 255))
    else:
        newData.append((0,0,0,0))

img.putdata(newData)
img.save("Images/axial_mask.png", "PNG")

In [3]:
#This overlays the positron image on top of the CT, usees the mask to make any black pixels in the mask transparent on the 
# positron image, all others retain their original value. Whats left is the pixels with low annihilation counts to be 
# transparent and high values to appear bright, overlayed on top of CT

import cv2
 
# Read the images
foreground = cv2.imread("Images/Pos_axial.png")
background = cv2.imread("Images/Patient_axial.png")
alpha = cv2.imread("Images/Pos_mask.png")
#cv2.imshow("outImg", background/255)
cv2.waitKey(0)
background = cv2.resize(background, (foreground.shape[1], foreground.shape[0]))
    
# Convert uint8 to float
foreground = foreground.astype(float)
#cv2.imshow("outImg", foreground/255)
cv2.waitKey(0)
background = background.astype(float)
#cv2.imshow("outImg", background/255)
cv2.waitKey(0)

# Normalize the alpha mask to keep intensity between 0 and 1
alpha = alpha.astype(float)/255
#cv2.imshow("outImg", alpha/255)
cv2.waitKey(0)

print(len(background))
print(len(foreground))
print(len(alpha))
    
# Multiply the foreground with the alpha matte
foreground = cv2.multiply(alpha, foreground)
#cv2.imshow("outImg", foreground/255)
cv2.waitKey(0)

# Multiply the background with ( 1 - alpha )
background = cv2.multiply(1.0 - alpha, background)
#cv2.imshow("outImg", background/255)
cv2.waitKey(0)

# Add the masked foreground and background.
outImage = cv2.add(foreground, background)
 
# Display image
cv2.imshow("outImg", outImage/255)
cv2.waitKey(0)

1056
1056
1056


-1

In [4]:
def make_mask(image, RGB_limit):
    from PIL import Image

    img = Image.open(image)
    img = img.convert("RGBA")
    datas = img.getdata()

    RGB1 = RGB_limit
    RGB2 = RGB_limit
    RGB3 = RGB_limit

    newData = []
    for item in datas:
        if item[0] >= RGB1 or item[1] >= RGB2 or item[2] >= RGB3:
            newData.append((255, 255, 255, 255))
        else:
            newData.append((0,0,0,0))

    img.putdata(newData)
    return img

def overlay(background,foreground,alpha):
    
    #takes in two image filenames "img1.png"
    
    #This overlays the positron image on top of the CT, usees the mask to make any black pixels in the mask transparent on the 
    # positron image, all others retain their original value. Whats left is the pixels with low annihilation counts to be 
    # transparent and high values to appear bright, overlayed on top of CT

    import cv2
 
    # Read the images
    foreground = cv2.imread(foreground)
    background = cv2.imread(background)
    alpha = cv2.imread(alpha)
    #cv2.imshow("outImg", background/255)
    cv2.waitKey(0)
    background = cv2.resize(background, (foreground.shape[1], foreground.shape[0]))
    
    # Convert uint8 to float
    foreground = foreground.astype(float)
    #cv2.imshow("outImg", foreground/255)
    cv2.waitKey(0)
    background = background.astype(float)
    #cv2.imshow("outImg", background/255)
    cv2.waitKey(0)

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha = alpha.astype(float)/255
    #cv2.imshow("outImg", alpha)
    cv2.waitKey(0)
    
    # Multiply the foreground with the alpha matte
    foreground = cv2.multiply(alpha, foreground)
    #cv2.imshow("outImg", foreground/255)
    cv2.waitKey(0)

    # Multiply the background with ( 1 - alpha )
    background = cv2.multiply(1-alpha, background)
    #cv2.imshow("outImg", background/255)
    cv2.waitKey(0)

    # Add the masked foreground and background.
    outImage = cv2.add(foreground, background)
 
    return outImage

In [5]:
axial_overlay = overlay("Images/Patient_axial.png","Images/Pos_axial.png","Images/axial_mask.png")

cv2.imwrite("Images/axial_overlay.png",axial_overlay)

True

In [6]:
#repeat this for other slices
axial_mask = make_mask("Images/Pos_axial.png",90)
axial_mask.save("Images/axial_mask.png", "PNG")

axial_overlay2 = overlay("Images/Patient_axial.png","Images/Pos_axial.png","Images/axial_mask.png")

cv2.imwrite("Images/axial_overlay2.png",axial_overlay2)


cor_mask = make_mask("Images/Pos_coronal.png",20)
cor_mask.save("Images/coronal_mask.png", "PNG")

cor_overlay = overlay("Images/Patient_coronal.png","Images/Pos_coronal.png","Images/coronal_mask.png")

cv2.imwrite("Images/cor_overlay.png",cor_overlay)


sag_mask = make_mask("Images/Pos_sagittal.png",30)
sag_mask.save("Images/sagittal_mask.png", "PNG")

sag_overlay = overlay("Images/Patient_sagital.png","Images/Pos_sagittal.png","Images/sagittal_mask.png")

cv2.imwrite("Images/sag_overlay.png",sag_overlay)

True

In [9]:
#Repeat this for reconstructions

#general idea is to export coordinates of all reconstuction points to MATLAB, then voxelize with 2mm spacing,
# save as .mat array, import back into python, convert to numpy array and continue as with true map
#
#getting the images to be the same aspect ratio will be tricky

axial_recon_mask = make_mask("Images/test_axial_recon_overlay.png",30)
axial_recon_mask.save("Images/axial_recon_mask.png", "PNG")

axial_recon_overlay = overlay("Images/Patient_axial.png","Images/test_axial_recon_overlay.png","Images/axial_recon_mask.png")

cv2.imwrite("Images/axial_recon_overlay.png",axial_recon_overlay)

sag_recon_mask = make_mask("Images/test_sag_recon_overlay.png",90)
sag_recon_mask.save("Images/sag_recon_mask.png", "PNG")

sag_recon_overlay = overlay("Images/Patient_sagital.png","Images/test_sag_recon_overlay.png","Images/sag_recon_mask.png")


cv2.imwrite("Images/sag_recon_overlay.png",sag_recon_overlay)

cor_recon_mask = make_mask("Images/test_cor_recon_overlay_flipped.png",90)
cor_recon_mask.save("Images/cor_recon_mask.png", "PNG")
cor_recon_overlay = overlay("Images/Patient_coronal.png","Images/test_cor_recon_overlay.png","Images/cor_recon_mask.png")


cv2.imwrite("Images/cor_recon_overlay.png",cor_recon_overlay)


True

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

#get an image that wasn't stitched together for 2cm sup

CT_2cmsup = sitk.GetArrayFromImage(sitk.ReadImage("Images/QUASAR/QUASAR_2cmsup_CT_cor_crop.png", sitk.sitkFloat32))

plt.figure(figsize=(8,25))
plt.gray()
plt.imshow(CT_2cmsup,cmap='gist_gray'), plt.axis('off')
    # use plt.savefig(...) here if you want to save the images as .jpg, e.g.,
#plt.colorbar()
plt.show()

#save as cropped image, fed to code below to make real image to overlay with positron map

In [ ]:
#Repeat above for QUASAR images


#centered axial
QUASAR_axial_mask = make_mask("Images/QUASAR/QUASAR_axial_pos.png",20)
QUASAR_axial_mask.save("Images/QUASAR/QUASAR_axial_mask.png", "PNG")

QUASAR_axial_overlay = overlay("Images/QUASAR/QUASAR_axial_CT.png","Images/QUASAR/QUASAR_axial_pos.png","Images/QUASAR/QUASAR_axial_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_axial_overlay.png",QUASAR_axial_overlay)


#centered coronal
QUASAR_cor_mask = make_mask("Images/QUASAR/QUASAR_cor_pos.png",90)
QUASAR_cor_mask.save("Images/QUASAR/QUASAR_cor_mask.png", "PNG")

QUASAR_cor_overlay = overlay("Images/QUASAR/QUASAR_cor_CT.png","Images/QUASAR/QUASAR_cor_pos.png","Images/QUASAR/QUASAR_cor_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_cor_overlay.png",QUASAR_cor_overlay)

Original_Image = Image.open("Images/QUASAR/QUASAR_cor_overlay.png") 
Original_Image.show() 

    
# Rotate Image By 180 Degree 
#rotated_image1 = Original_Image.rotate(180)
#flip_img = Original_Image.transpose(Image.FLIP_TOP_BOTTOM)
#flip_img.show() 

#centered sagittal
QUASAR_sag_mask = make_mask("Images/QUASAR/QUASAR_sag_pos.png",20)
QUASAR_sag_mask.save("Images/QUASAR/QUASAR_sag_mask.png", "PNG")

QUASAR_sag_overlay = overlay("Images/QUASAR/QUASAR_sag_CT.png","Images/QUASAR/QUASAR_sag_pos.png","Images/QUASAR/QUASAR_sag_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_sag_overlay.png",QUASAR_sag_overlay)


#superior coronal
QUASAR_sup_cor_mask = make_mask("Images/QUASAR/QUASAR_sup_pos_cor.png",90)
QUASAR_sup_cor_mask.save("Images/QUASAR/QUASAR_sup_cor_mask.png", "PNG")

QUASAR_sup_cor_overlay = overlay("Images/QUASAR/QUASAR_sup_CT_cor.png","Images/QUASAR/QUASAR_sup_pos_cor.png","Images/QUASAR/QUASAR_sup_cor_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_sup_cor_overlay.png",QUASAR_sup_cor_overlay)

Original_Image = Image.open("Images/QUASAR/QUASAR_sup_cor_overlay.png") 
Original_Image.show() 

    
# Rotate Image By 180 Degree 
#rotated_image1 = Original_Image.rotate(180)
#flip_img = Original_Image.transpose(Image.FLIP_TOP_BOTTOM)
#flip_img.show() 

#inferior coronal
QUASAR_inf_cor_mask = make_mask("Images/QUASAR/QUASAR_inf_pos_cor.png",90)
QUASAR_inf_cor_mask.save("Images/QUASAR/QUASAR_inf_cor_mask.png", "PNG")

QUASAR_inf_cor_overlay = overlay("Images/QUASAR/QUASAR_inf_CT_cor.png","Images/QUASAR/QUASAR_inf_pos_cor.png","Images/QUASAR/QUASAR_inf_cor_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_inf_cor_overlay.png",QUASAR_inf_cor_overlay)

Original_Image = Image.open("Images/QUASAR/QUASAR_inf_cor_overlay.png") 
Original_Image.show() 

# Rotate Image By 180 Degree 
#rotated_image1 = Original_Image.rotate(180)
#flip_img = Original_Image.transpose(Image.FLIP_TOP_BOTTOM)
#flip_img.show() 
#2cm superior coronal

#Try to displace superior CT by 1cm in lng direction and overlay, using a modified image feels wrong

CTsup = cv2.imread("Images/QUASAR/QUASAR_sup_CT_cor.png")
CT2cmsup = cv2.imread("Images/QUASAR/QUASAR_2cmsup_CT_cor.png")
CT2cmsup = cv2.resize(CT2cmsup, (CTsup.shape[1], CTsup.shape[0]))
cv2.imshow("outImg", CT2cmsup/255)

cv2.imwrite("Images/QUASAR/QUASAR_2cmsup_CT_cor.png",CT2cmsup)

                      
QUASAR_2cmsup_cor_mask = make_mask("Images/QUASAR/QUASAR_2cmsup_pos_cor.png",90)
QUASAR_2cmsup_cor_mask.save("Images/QUASAR/QUASAR_2cmsup_cor_mask.png", "PNG")

QUASAR_2cmsup_cor_overlay = overlay("Images/QUASAR/QUASAR_2cmsup_CT_cor.png","Images/QUASAR/QUASAR_2cmsup_pos_cor.png","Images/QUASAR/QUASAR_2cmsup_cor_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_2cmsup_cor_overlay.png",QUASAR_2cmsup_cor_overlay)

Original_Image = Image.open("Images/QUASAR/QUASAR_2cmsup_cor_overlay.png") 
  
# Rotate Image By 180 Degree 
#rotated_image1 = Original_Image.rotate(180)
#flip_img = Original_Image.transpose(Image.FLIP_TOP_BOTTOM)
Original_Image.show() 


In [ ]:
#final thing is to repeat for QUASAR reconstructions which should be identical to process for patient, aside from 
# getting new image bounds for QUASAR CT

axial_recon_mask = make_mask("Images/QUASAR/bad_format/QUASAR_axial_recon.png",200)
axial_recon_mask.save("Images/QUASAR/QUASAR_axial_recon_mask.png", "PNG")

axial_recon_overlay = overlay("Images/QUASAR/bad_format/QUASAR_axial_CT.png","Images/QUASAR/bad_format/QUASAR_axial_recon.png","Images/QUASAR/QUASAR_axial_recon_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_axial_recon_overlay.png",axial_recon_overlay)

cor_center_recon_mask = make_mask("Images/QUASAR/bad_format/QUASAR_cor_center_recon.png",70)
cor_center_recon_mask.save("Images/QUASAR/QUASAR_cor_center_recon_mask.png", "PNG")

cor_center_recon_overlay = overlay("Images/QUASAR/bad_format/QUASAR_cor_CT.png","Images/QUASAR/bad_format/QUASAR_cor_center_recon.png","Images/QUASAR/QUASAR_cor_center_recon_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_cor_center_recon_overlay.png",cor_center_recon_overlay)

cor_sup_recon_mask = make_mask("Images/QUASAR/bad_format/QUASAR_cor_sup_recon.png",70)
cor_sup_recon_mask.save("Images/QUASAR/QUASAR_cor_sup_recon_mask.png", "PNG")

cor_sup_recon_overlay = overlay("Images/QUASAR/bad_format/QUASAR_cor_sup_CT.png","Images/QUASAR/bad_format/QUASAR_cor_sup_recon.png","Images/QUASAR/QUASAR_cor_sup_recon_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_cor_sup_recon_overlay.png",cor_sup_recon_overlay)

cor_2cmsup_recon_mask = make_mask("Images/QUASAR/bad_format/QUASAR_cor_2cmsup_recon.png",70)
cor_2cmsup_recon_mask.save("Images/QUASAR/QUASAR_cor_2cmsup_recon_mask.png", "PNG")

cor_2cmsup_recon_overlay = overlay("Images/QUASAR/bad_format/QUASAR_cor_2cmsup_CT.png","Images/QUASAR/bad_format/QUASAR_cor_2cmsup_recon.png","Images/QUASAR/QUASAR_cor_2cmsup_recon_mask.png")

cv2.imwrite("Images/QUASAR/QUASAR_cor_2cmsup_recon_overlay.png",cor_2cmsup_recon_overlay)

